## Librerías

In [9]:
%run "../recurrentes.ipynb"

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [10]:
%run "../funciones.ipynb"

In [11]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [28]:
#1 - Carga del dataset. Opciones: 
# dataset_lags_clase_ternaria_l : dataset con lags y clase ternaria
# dataset_clase_ternaria_l: dataset sin lags y clase ternaria

df_train = pd.read_parquet(dataset_clase_ternaria_l)

In [29]:
df_train.shape

(4735593, 155)

## Preparación del dataset para el Voting GD

In [32]:
# 2- Para el caso de que el dataset sea con feature engineering incluido
if 'T_Visa_normal' in df_train.columns and 'T_Master_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)

In [ ]:
# 3- parametros para calcular ganancia del modelo
ganancia_acierto = 273000
costo_estimulo = 7000

data = df_train
data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

# 4 - binarización de la clase ternaria

data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

# 5 Especificar mes de train y test

df_train = data[data['foto_mes']<=202106]
df_test = data[data['foto_mes']==202108]


clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train.loc[X_train.index, 'clase_peso']
w_test = df_test.loc[X_test.index, 'clase_peso']


In [ ]:
# 6 Ingresar mejores parámetros optuna. Remplazar los parámetros con mejores datos de optuna corridos en VM
best_params = {'num_leaves': 40,
 'learning_rate': 0.022596031021514126,
 'min_data_in_leaf': 1692,
 'feature_fraction': 0.8060065372089812,
 'bagging_fraction': 0.8663914349386255}

# best_iter = study.best_trial.user_attrs["best_iter"]


## Semillero

In [ ]:


train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)

df_voting = pd.DataFrame()

for semilla in numeros_random:
    params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': 0,
    'learning_rate': 0,
    'min_data_in_leaf': 0,
    'feature_fraction': 0,
    'bagging_fraction': 0,
    'seed': semilla,
    'verbose': 0
}
    #actualización con valores optuna
    params.update(best_params)
    
    if params['num_leaves'] == 0:
        print ('Error falta actualización de parámetros')
        break
    
    model = lgb.train(params,
                    train_data,
                    num_boost_round=299)
    
    df_voting[f'semilla_{semilla}'] = model.predict(X_test)

df_voting['promedio'] = df_voting.mean(axis=1)
    

ValueError: Input data must be 2 dimensional and non empty.

## Predicción

In [ ]:
y_pred = df_voting['promedio']

## Kaggle

### Script para subida individual 

In [ ]:
df_resultado = pd.DataFrame({
    'numero_de_cliente': X_test['numero_de_cliente'],
    'Predicted': y_pred
}, index = X_test.index)

In [ ]:

numero = '002'
entrega = f'entrega_{numero}.csv'
path = entregas_l
archivos = glob.glob(os.path.join(path, '*'))
archivos.sort(key=os.path.getmtime, reverse=True)
ultimo_archivo = archivos[0] if archivos else None
if ultimo_archivo:
    nombre_archivo = os.path.basename(ultimo_archivo)
    numero = ''.join(filter(str.isdigit, nombre_archivo))
    print(f"El último archivo entregado es: {nombre_archivo}")
    print(f"El número extraído es: {numero}")
else:
    print("No se encontraron archivos en la carpeta.")


 

In [ ]:

# numero = '073'
# entrega = f'entrega_{numero}.csv'

In [ ]:
# if os.path.exists(path + entrega):
#     print("El archivo ya existe")
# else:
#     entrega_final = os.path.join(path, entrega)
#     df_resultado.to_csv(entrega_final, index=False)
    


In [ ]:
# cantidad_columnas = df_train.shape[1]
# message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM,  mejor iteracion: {best_iter}, archivo: {entrega_final},"

In [ ]:

# entrega_final = os.path.join(path, entrega)
# entrega_final
# competition= 'dm-ey-f-2024-segunda'

In [ ]:
# kaggle.api.competition_submit(competition=competition, file_name= entrega_final, message=message, quiet=False)

### Subida en serie a Kaggle

In [ ]:


predicciones = y_pred_lgm

X_test['Probabilidad'] = predicciones

tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

cortes = range(9000,14000,100)

num_subida_kaggle = 2
for envios in cortes:
    
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
    print("Cantidad de clientes {}".format(envios))
    
    nombre_archivo = 'entrega_0{}.csv'.format(num_subida_kaggle)
    entrega_final = os.path.join(path, nombre_archivo)
    resultados.to_csv(entrega_final, index=False)
    
    
    cantidad_columnas = df_train.shape[1]
    message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM, mejores parametros: {best_params}, mejor iteracion: {best_iter}, archivo: {entrega_final}, punto de corte: {envios}, optimizado con optuna: {study_name}"
    
    num_subida_kaggle += 1
    
    
    
    entrega_final = os.path.join(path, nombre_archivo)   
    competencia = 'dm-ey-f-2024-primera'
    try:
        api.competition_submit(file_name=entrega_final,message=message,competition=competencia)
    except:
        print(f"Numero máximo de envios, último envio ={num_subida_kaggle}")

In [ ]:
# df_train.shape

In [ ]:
# nombre_modelo = 'lgbm_e_en_abril_p_en_junio_451_features.txt'
# model.save_model(rf"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\modelos_lgbm\{nombre_modelo}")